Ahora para este script me dare la tarea de entrenar varios tipos de modelos de ML para ver cual mantiene un mejor rendimiento y mejores resultados.

In [212]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV


train_set = pd.read_csv(filepath_or_buffer=r"titanic\train.csv", sep=',', encoding='utf-8')
company_list = []
for i in train_set['SibSp']:
    if i == 0:
        company_list.append(0)
    elif 1<=i<=4:
        company_list.append(1)
    elif i>4:
        company_list.append(2)
train_set['Company'] = pd.DataFrame(company_list)

Ahora empezaremos con un modelo de Regresion Logistica.

Para ello necesitamos aplicar el conocimiento adquirido del EDA y transformal los datos para que se ajuste a la forma en que se usa en el modelo LogisticRegression(), para eso utilizaremos LabelEncoder() para convertir las categorias tipo string a numericas y ademas retiraremos las columnas que no nos seran de utilidad para nuestro modelo por su baja correlacion con la variable objetivo.

Primero eliminamos 'Name' y 'Ticket' porque no aportaban nada a la variable objetivo y tambien la columna 'Cabin' ya que presentaba una gran cantidad de nulo que no seria facil tratar

In [213]:
train_set = train_set[['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']]
train_set

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Company
0,0,3,male,22.0,1,0,7.2500,S,1
1,1,1,female,38.0,1,0,71.2833,C,1
2,1,3,female,26.0,0,0,7.9250,S,0
3,1,1,female,35.0,1,0,53.1000,S,1
4,0,3,male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0
887,1,1,female,19.0,0,0,30.0000,S,0
888,0,3,female,NaN,1,2,23.4500,S,1
889,1,1,male,26.0,0,0,30.0000,C,0


Ahora hay que tranformar 'Sex', 'Age' y 'Embarked' para hacer tratado de lo nulos y transformacion en valores numericos

In [214]:
label_encoder = LabelEncoder()

train_set['Sex'] = label_encoder.fit_transform(train_set['Sex'])
train_set['Embarked'] = label_encoder.fit_transform(train_set['Embarked'])

Para la columna 'Age' necesitaremos que los valores nulos sean tratados de alguna manera, la columna tiene pocos por lo cual es rentable manejarla.

En este caso rellenaremos los resultados nulos con el promedio de la edad para que afecte en lo minimo.

In [215]:
train_set

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Company
0,0,3,1,22.0,1,0,7.2500,2,1
1,1,1,0,38.0,1,0,71.2833,0,1
2,1,3,0,26.0,0,0,7.9250,2,0
3,1,1,0,35.0,1,0,53.1000,2,1
4,0,3,1,35.0,0,0,8.0500,2,0
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2,0
887,1,1,0,19.0,0,0,30.0000,2,0
888,0,3,0,NaN,1,2,23.4500,2,1
889,1,1,1,26.0,0,0,30.0000,0,0


In [216]:
train_set['Age'].mean()

29.69911764705882

In [217]:
train_set['Age'][train_set['Age'].isnull() == True] = 29.70

C:\Users\Jose J\AppData\Local\Temp\ipykernel_9044\1953619782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['Age'][train_set['Age'].isnull() == True] = 29.70


Ahora enfrentamo un ligero inconveniente que podria afectar el rendimiento del modelo de regresion logistica, el cual es la diferencias entre las magnitudes de las distintas variables independientes, por lo tanto se hara uso de el metodo de escalado MinMaxScaler con el cual se pueden presentar todos los datos en un rango de -1 a 1 manteniendo la misma distancia relativa entra cada uno.

In [218]:
scaler = MinMaxScaler(feature_range=(-1, 1))

train_set[[
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']] = pd.DataFrame(scaler.fit_transform(train_set[[
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']]))
train_set

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Company
0,0,1.0,1.0,-0.457653,-0.75,-1.000000,-0.971698,0.333333,0.0
1,1,-1.0,-1.0,-0.055542,-0.75,-1.000000,-0.721729,-1.000000,0.0
2,1,1.0,-1.0,-0.357125,-1.00,-1.000000,-0.969063,0.333333,-1.0
3,1,-1.0,-1.0,-0.130937,-0.75,-1.000000,-0.792711,0.333333,0.0
4,0,1.0,1.0,-0.130937,-1.00,-1.000000,-0.968575,0.333333,-1.0
...,...,...,...,...,...,...,...,...,...
886,0,0.0,1.0,-0.331993,-1.00,-1.000000,-0.949251,0.333333,-1.0
887,1,-1.0,-1.0,-0.533049,-1.00,-1.000000,-0.882888,0.333333,-1.0
888,0,1.0,-1.0,-0.264137,-0.75,-0.333333,-0.908457,0.333333,0.0
889,1,-1.0,1.0,-0.357125,-1.00,-1.000000,-0.882888,-1.000000,-1.0


Ahora ya tenemos entrenado el modelo de Regresion Logistica, ahora tenemos que preparar los datos de testeo de igual manera que los de entrenamiento.

In [219]:
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(train_set[[
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']], train_set['Survived'])

LogisticRegression()

Cargamos el archivo test.csv el cual contiene los datos para predecir y evaluar el modelo

In [220]:
test_set = pd.read_csv(r'titanic\test.csv', sep=',', encoding='utf-8')
test_set['Company'] = [1 if i>0 else 0 for i in test_set['SibSp']]

Ahora necesitamos igual que con el train_set eliminar las columnas irrelevantes.

In [221]:
test_set = test_set[['PassengerId',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']]
test_set

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Company
0,892,3,male,34.5,0,0,7.8292,Q,0
1,893,3,female,47.0,1,0,7.0000,S,1
2,894,2,male,62.0,0,0,9.6875,Q,0
3,895,3,male,27.0,0,0,8.6625,S,0
4,896,3,female,22.0,1,1,12.2875,S,1
...,...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S,0
414,1306,1,female,39.0,0,0,108.9000,C,0
415,1307,3,male,38.5,0,0,7.2500,S,0
416,1308,3,male,NaN,0,0,8.0500,S,0


Para este archivo podemos notar presencia de nulos, los cuales deberan ser tratados en su defecto con su media.

In [222]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          332 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         417 non-null    float64
 7   Embarked     418 non-null    object 
 8   Company      418 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 29.5+ KB


In [223]:
test_set['Age'][test_set['Age'].isnull() == True] = test_set['Age'].mean().__round__(1)

Ahora que tenemos los datos tipo string limpios podemos utilizar el mismo label encoder para transformar igualmente los valores tipo string en numero enteros

In [224]:
test_set['Sex'] = label_encoder.fit_transform(test_set['Sex'])
test_set['Embarked'] = label_encoder.fit_transform(test_set['Embarked'])

Se rellena en la columna 'Fare' el valor nulo por su media

In [225]:
test_set['Fare'][test_set['Fare'].isnull() == True] = test_set['Fare'].mean()

C:\Users\Jose J\AppData\Local\Temp\ipykernel_9044\3159694245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['Fare'][test_set['Fare'].isnull() == True] = test_set['Fare'].mean()


Ahora transformamos los valores a una escala relativa. Igual que con el train_set

In [226]:
test_set_test = pd.DataFrame(scaler.fit_transform(test_set[[
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']]))
test_set_test

,0,1,2,3,4,5,6,7
0,1.0,1.0,-0.094554,-1.00,-1.000000,-0.969437,0.0,-1.0
1,1.0,-1.0,0.235131,-0.75,-1.000000,-0.972674,1.0,1.0
2,0.0,1.0,0.630753,-1.00,-1.000000,-0.962183,0.0,-1.0
3,1.0,1.0,-0.292364,-1.00,-1.000000,-0.966184,1.0,-1.0
4,1.0,-1.0,-0.424238,-0.75,-0.777778,-0.952033,1.0,1.0
...,...,...,...,...,...,...,...,...
413,1.0,1.0,-0.205328,-1.00,-1.000000,-0.968575,1.0,-1.0
414,-1.0,-1.0,0.024133,-1.00,-1.000000,-0.574883,-1.0,-1.0
415,1.0,1.0,0.010946,-1.00,-1.000000,-0.971698,1.0,-1.0
416,1.0,1.0,-0.205328,-1.00,-1.000000,-0.968575,1.0,-1.0


Finalmente predecimos con el metodo predict con nuestro modelo entrenado previamente

In [227]:
logistic_regresion_predict = pd.DataFrame(logistic_regression_model.predict(test_set_test))

e:\Cosas Intaladas Aqui ltsc\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [228]:
logistic_regresion_predict_response = pd.concat((test_set['PassengerId'],logistic_regresion_predict),axis=1)

In [229]:
logistic_regresion_predict_response = logistic_regresion_predict_response.rename(columns={0: 'Survived'})
logistic_regresion_predict_response.to_csv('Response.csv', sep=',', encoding='utf-8',index=False)

Y con nuestro ultimo codigo presentamos la respuesta a evaluar, este mismo lo estaremos evaluando con los resultados obtenidos de Kaggle.

Para el cual inicialmente nos presenta una un 0.77751 de precision de 1, ahora ahondaremos el tema de los parametros que puedieran mejorar el rendimiento de el modelo de regresion logistica, para ello utilizaremos GridSearchCV para evaluar diferentes parametros

In [230]:
def model_x_CV(model, param_grid, x_train, y_train):
    grid_search_cv = GridSearchCV(model,param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search_cv.fit(x_train, y_train)
    best_params = grid_search_cv.best_params_
    return best_params

In [231]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.03,0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 500]
}

model = LogisticRegression()
x_train = train_set[[
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Company']]
y_train = train_set['Survived']

model_x_CV(model=model,param_grid=param_grid,x_train=x_train, y_train=y_train)

{'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}

Ahora podemos volver a entrenar un modelo nuevo con estos parametros para observar la diferencia entre ellos.

In [232]:
def model_trainer(model,x_train, y_train, x_test, filename: str):
    model.fit(x_train, y_train)
    model_predict = pd.DataFrame(model.predict(x_test))

    model_response = pd.concat((test_set['PassengerId'],model_predict),axis=1)

    model_response = model_response.rename(columns={0: 'Survived'})

    model_response.to_csv(filename, sep=',', encoding='utf-8', index=False)
    return print('Archivo guardado como: ', filename)

In [233]:
logistic_regression_model_params_fixed = LogisticRegression(C=1, max_iter=100, penalty='l2', solver='liblinear')
model_trainer(logistic_regression_model_params_fixed,x_train=x_train, y_train=y_train, x_test=test_set_test,filename='Response_params_fixed.csv')

Archivo guardado como:  Response_params_fixed.csv


e:\Cosas Intaladas Aqui ltsc\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Este modelo nos a dado una precision ligeramente superior, con un 0.7799 de precision casi 0.78, por lo tanto llegamos al fondo de la optimizacion de el modelo de regresion logistica, por lo tanto ya es hora de evaluar otro modelo.

Para esta ocasion utilizaremos el modelo de RandomForest y de ahora en adelante para agilizar el preceso comenzaremos utilizando la validacion cruzada para encontrar los mejores parametros.

In [234]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}
model_x_CV(random_forest, param_grid=param_grid, x_train=x_train, y_train=y_train)

{'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}

Podemos observar los mejores parametros para nuestro modelo de random forest

In [235]:
random_forest = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=50)

model_trainer(random_forest,x_train=x_train, y_train=y_train, x_test=test_set_test, filename='Response_RF_CV.csv')

Archivo guardado como:  Response_RF_CV.csv


e:\Cosas Intaladas Aqui ltsc\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Ahora al evaluar el modelo nos da una precision del 0.75837 por lo tanto se nota que hubo una disminucion con respecto al modelo anterior pero ronda por los mismos valores

Dado que aun no subimos la precision debemos probar con otro modelo de aprendizaje, en este caso usaremos el modelo de SVM

Haciendo uso de la funcion de GridSearch podemos obtener los mejores parametros para hacer en entrenamiento.

In [236]:
from sklearn.svm import SVC
svc_model = SVC()
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'],
}
print(model_x_CV(svc_model,param_grid=param_grid,x_train=x_train, y_train=y_train))

{'C': 10, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}


Entrenamos el modelo.

In [237]:
svc_model = SVC(C=10, degree=3, gamma='scale', kernel='poly')
model_trainer(svc_model, x_train=x_train, y_train=y_train, x_test=test_set_test, filename='Response_SVM_CV.csv')

Archivo guardado como:  Response_SVM_CV.csv


e:\Cosas Intaladas Aqui ltsc\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Luego de haber entrenado y testeado arroja una presicion del 0.76794 por lo tanto es mejor que el random forest pero peor que la regresion logistica.

Ahora procederemos en el modelo KNN

In [238]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [3, 5, 7, 9,12, 15, 20, 30, 40],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}
model_x_CV(knn_model,param_grid=param_grid,x_train=x_train,y_train=y_train)

{'n_neighbors': 12, 'p': 1, 'weights': 'uniform'}

Ahora entrenamos nuestro modelo con los nuevos parametros.

In [239]:
knn_model = KNeighborsClassifier(n_neighbors=12, p=2, weights='uniform')
model_trainer(knn_model, x_train=x_train, y_train=y_train, x_test=test_set_test, filename='Response_KNN_CV.csv')

Archivo guardado como:  Response_KNN_CV.csv


e:\Cosas Intaladas Aqui ltsc\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


El modelo presente tuvo una precision del 0.77751, lo cual hasta ahora no presenta mejora alguna con respecto al modelo de regresion logistica pero sirve para dar por finalizado las distintas pruebas de diferentes modelos de aprendizaje para esta data de pasajeros del Titanic.

Podemos concluir que dicho modelo de Regresion Logistica fue el que mejor resultados aporto con sus respectivos parametros